## STEP 1 Install Darknet


# Tutorial: Training a Yolov4 Object Detector with poker cards images



In this tutorial we will train an object detector using the Yolo v4 model. This model will run on Ultra96v2 board.

The model is pretrained on the COCO dataset. The framework used for training is Darknet.
Will run through the following steps:


*   Install the libraries (Darknet , etc.)
*   Clone the github repo and replace the repo training data with your data (from google drive or from own repo - which is faster)
*   Train the model on the new images


You can make a copy of this tutorial: File-> Save a copy in Drive

Note: the model training can be run with the repo images of 6 poker cards.If you choose to skip the customization part for your own images just to see how the training and the rest of the steps work.

In [3]:
%%capture
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
import re
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet


In [4]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
#utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  


## Clone a reference/blueprint repo
This repo provide the flow from yolov4 model training to deployment.
In order to train on your own data, this repo can be used as a blueprint. 
Changes to some files need to be made according to own data, but it is nothing complicated.

In [6]:
#!pwd
%cd ..
!git clone https://github.com/brightcome/kids-math-study-ai-mate-on-ultra96v2
!ls


/content
Cloning into 'kids-math-study-ai-mate-on-ultra96v2'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 120 (delta 4), reused 0 (delta 0), pack-reused 109
Receiving objects: 100% (120/120), 224.89 MiB | 50.54 MiB/s, done.
Resolving deltas: 100% (9/9), done.
darknet  kids-math-study-ai-mate-on-ultra96v2  sample_data


### Now go to the file explorer on the left of the colab notebook and click on content then on the "kids-math-study-ai-mate-on-ultra96v2" folder.
Here you will see a folder and a few files.
The `train_images` folder contains both images AND the associated annotations in yolo format. 

(to annotate own images use https://github.com/tzutalin/labelImg and save annotations in yolo format)
The dk_files folder contains training related files and command.
* the 'obj.data' file contains paths for files used by Darknet and the number of classes to train the yolo model.

* the 'obj.names' file has the labels of the objects to be detected. the number of objects gives the number of classes in the 'obj.data' file.

* the train_list.txt contains the path to each image to be used in training. similarly, valid_list.txt contains the path to each image for testing. when not many images are available for training (a few hundred), use all images for training for better model performance. As a simple demo , I only have 40 images for training and validation. You should take pictures by your own and add more pictures into the folder.

The dk_model folder contains darknet yolov4 model file.
* the yolov4-leaky_best.weights.7z.001 are the COCO pretrained weights. Please unzip the package files. 

* the yolov4-leaky_obj.cfg are is the yolov4 configuration file that tells Darknet the model parameters and structure. the cfg file has been changed according to my demo requirement- 6 classes.

So you should change according to your data and classes, if needed.


# How to use your own images
In order to use your own annotated images, follow the steps below. If you want to train on the images already in the repo, skip this section and jump to the Train model section.

## Let's start with the images and annotations. I will use my images as example.
We copy the images on our google drive in a folder. My gdrive folder is named medmask_yolo. Then we mount the gdrive in colab and copy the images and annotations in the obj folder.

clone one repo for the poker training image reference.

In [8]:
!git clone https://github.com/AbinMM/TensorFlow-Poker-Object-Detection
!ls

Cloning into 'TensorFlow-Poker-Object-Detection'...
remote: Enumerating objects: 960, done.
remote: Total 960 (delta 0), reused 0 (delta 0), pack-reused 960
Receiving objects: 100% (960/960), 57.13 MiB | 39.48 MiB/s, done.
Resolving deltas: 100% (466/466), done.


copy the images to our Repo

In [12]:
!rm -rf ./kids-math-study-ai-mate-on-ultra96v2/train_images/*.jpg
!rm -rf ./kids-math-study-ai-mate-on-ultra96v2/train_images/*.txt
!rm -rf ./kids-math-study-ai-mate-on-ultra96v2/train_images/*.labels
!cp -rf ./TensorFlow-Poker-Object-Detection/images/test ./kids-math-study-ai-mate-on-ultra96v2/train_images
!cp -rf ./TensorFlow-Poker-Object-Detection/images/train ./kids-math-study-ai-mate-on-ultra96v2/train_images

convert the label files from xml to yolo format.run the covert scripts

In [15]:
%cd kids-math-study-ai-mate-on-ultra96v2/train_images/
!python label_xml2yolo.py

/content/kids-math-study-ai-mate-on-ultra96v2/train_images
train/cam_image30
train/IMG_2523
train/cam_image16
train/IMG_2513
train/IMG_2487
train/IMG_2514
train/cam_image36
train/IMG_2402
train/IMG_2466
train/IMG_2406
train/IMG_2492
train/IMG_2502
train/IMG_2660
train/IMG_2509
train/IMG_2398
train/IMG_2467
train/IMG_2611
train/cam_image23
train/IMG_2596
train/cam_image29
train/IMG_2428
train/IMG_2421
train/IMG_2416
train/IMG_2494
train/IMG_2616
train/IMG_2442
train/IMG_2622
train/IMG_2543
train/IMG_2687
train/IMG_2400
train/cam_image44
train/IMG_2508
train/IMG_2524
train/IMG_2563
train/IMG_2632
train/IMG_2481
train/IMG_2390
train/IMG_2618
train/IMG_2464
train/IMG_2650
train/IMG_2447
train/IMG_2413
train/IMG_2673
train/IMG_2422
train/IMG_2415
train/IMG_2499
train/IMG_2386
train/IMG_2486
train/IMG_2536
train/IMG_2674
train/IMG_2668
train/IMG_2522
train/IMG_2549
train/IMG_2504
train/IMG_2517
train/IMG_2601
train/IMG_2630
train/IMG_2615
train/IMG_2434
train/IMG_2411
train/IMG_2626
train/IM

After the conversion is completed, transfer the images to darknet files folder.

In [19]:
#copy images to dk_files folder
#this can take a few minutes depending on dataset size. 
!pwd
#%cd ./dk_files/
!cp -rf ./train/*.* ../dk_files/obj
!cp -rf ./test/*.* ../dk_files/obj

/content/kids-math-study-ai-mate-on-ultra96v2/train_images


### generate image files list for training
The image convert package do NOT include list , so need generate.

In [ ]:
%cd /content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files/
!pwd
%cd ./train
!ls *.jpg > ../train_list.txt
%cd ../
%cd ./valid
!ls *.jpg > ../valid_list.txt
%cd ../

/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files
/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files
/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files/train
/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files
/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files/valid
/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files


extract the pre-trained yolov4 model weights

In [24]:
#！sudo apt install p7zip-full
! pwd
%cd ../dk_model
!7z x yolov4-leaky_best.weights.7z.001

/content/kids-math-study-ai-mate-on-ultra96v2/dk_files
/content/kids-math-study-ai-mate-on-ultra96v2/dk_model

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 24641535 bytes (24 MiB)

Extracting archive: yolov4-leaky_best.weights.7z.001
  0% 1 Open           --
Path = yolov4-leaky_best.weights.7z.001
Type = Split
Physical Size = 24641535
Volumes = 10
Total Physical Size = 234427775
----
Path = yolov4-leaky_best.weights.7z
Size = 234427775
--
Path = yolov4-leaky_best.weights.7z
Type = 7z
Physical Size = 234427775
Headers Size = 152
Method = LZMA:25
Solid = -
Blocks = 1

  0%      1%      2% - yolov4-leaky_best.weights                                  3% - yolov4-le

## Change the labels in obj.names to our current labels
Or you can directly use or upload my obj.names 

In [ ]:
labels_path = '/content/yolotinyv3_medmask_demo/obj.names'
#make a list of your labels
labels = ['ace','jack','king','nine','queen','ten']


with open(labels_path, 'w') as f:

    f.write('\n'.join(labels))

#check that the labels file is correct
!cat /content/yolotinyv3_medmask_demo/obj.names

mask
no mask

## Change the number of classes in obj.data.
The paths are relative so no change there as long as the folder/file structure/names are not changed.

In [ ]:
import re
objdata = '/content/yolotinyv3_medmask_demo/obj.data'
with open(objdata) as f:
    s = f.read()

#the number of classes is equal to the number of labels
num_classes = len(labels)   
s = re.sub('classes = \d*','classes = ' + str(num_classes),s)

with open(objdata, 'w') as f:
  f.write(s)
!cat /content/yolotinyv3_medmask_demo/obj.data

classes= 2
train  = /content/yolotinyv3_medmask_demo/train.txt
valid  = /content/yolotinyv3_medmask_demo/test.txt
names = /content/yolotinyv3_medmask_demo/obj.names
backup = backup/

## Now we need to create a train.txt and test.txt file for our images from `obj` folder.
We will use a script that splits the images based on a specified percentage 
and writes them in train.txt and test.txt files.

In [ ]:
%cd ../yolotinyv3_medmask_demo/

/content/yolotinyv3_medmask_demo


In [ ]:
#in this case we use 6 percent as the number of images for testing.
#double clicking on the train or test text files will open the file in a new
#tab in colab and display the content.
!python3 folder2textYolo.py 5 /content/yolotinyv3_medmask_demo/obj

Your image file extension is: .jpg
Number of images: 762
Number of images used for training 724
Number of images used for testing 38


## Last thing we need to change is the yolov4-leaky_obj.cfg file.
There are several parameters of importance here. They control various aspects of the training process. Let's print the first few lines and have a look


In [ ]:
!head -n 24 /content/yolotinyv3_medmask_demo/yolov4-leaky_obj.cfg 


[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=3200,3600
scales=.1,.1



### Let's go over the parameters above:
* The batch parameter dictates the batch size. That one generally remains at 64.
* The subdivisions dictates how many images are loaded into memory. A smaller number translates to faster training. We will use 12. If a CUDA out of memory error is triggered, the subdivisions should be increased to i.e. 16,24,32 or some other number. (64 is max) Unless training with a resolution higher than 416, there should be no problem with 12.
* The width and height are by default at 416. Another resolution one can try is 320, or 608.
* The next parameter we care about is max_batches. This determines how long the training process is. Its value should be at least around 2000 for every class used. So for 3 classes, at least 6000.
* The steps are calculated as a function of max_batches. The first value is 0.8 * max_batches and the second value 0.9 * max_batches. In this case it is 8000 * 0.8 and 8000 * 0.9. 

For more details about Yolo parameters have a look here:
https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects
There is a lot of material here, so make sure you scroll through if you have questions. Pretty much everything is well explained.




### Now that we see which parameters are of importance, let's modify them according to our dataset.

In [ ]:
# set the number of max_batches - min 2000 per class:
max_batch=4000
# calculate the 2 steps values:
step1 = 0.8 * max_batch
step2 = 0.9 * max_batch

# we also need to adjust the number of classes and a parameter called filter size 
# that are both is inside the model structure

# num_classes = len(labels)
num_filters = (num_classes + 5) * 3


cfg_file = '/content/yolotinyv3_medmask_demo/yolov4-leaky_obj.cfg'
# cfg_file = '/content/yolotinyv3_medmask_demo/yolov4-tiny.cfg'

with open(cfg_file) as f:
    s = f.read()
# (re.sub('[a-z]*@', 'ABC@', s))
s = re.sub('max_batches = \d*','max_batches = '+str(max_batch),s)
s = re.sub('steps=\d*,\d*','steps='+"{:.0f}".format(step1)+','+"{:.0f}".format(step2),s)
s = re.sub('classes=\d*','classes='+str(num_classes),s)
s = re.sub('pad=1\nfilters=\d*','pad=1\nfilters='+"{:.0f}".format(num_filters),s)
# pad=1\nfilters=\d\d
# s = re.sub('CUDNN=0','CUDNN=1',s)
# s = re.sub('OPENCV=0','OPENCV=1',s)

with open(cfg_file, 'w') as f:
  # s = re.sub('GPU=0','GPU=1',s)
  f.write(s)



Let's look again at the configuration 

In [ ]:
!head -n 24 /content/yolotinyv3_medmask_demo/yolov4-leaky_obj.cfg 


[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=3200,3600
scales=.1,.1



We also check if the number of classes and filters updated correctly

In [ ]:
!tail -n 64 /content/yolotinyv3_medmask_demo/yolov4-leaky_obj.cfg 


stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=21
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=2
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 8

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=21
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=2
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1


# Train the model

When you execute the following command, your model will start training.

You will have a log line per epoch. On each iteration you will see how your training is going.

In the `content/darknet/backup/` folder Darknet saves a few weights files:
* files that end in 1000,2000 etc are weights saved every 1000 batches
* best weights file are the weights that gave best results during training
* final weights file contains the final weights at the end of the training





## Start the model training

In [26]:
!pwd
%cd ../dk_files

/content/kids-math-study-ai-mate-on-ultra96v2/dk_model
/content/kids-math-study-ai-mate-on-ultra96v2/dk_files


In [30]:
!source ./train_yolov4_colab.sh

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 610, cudnn_half = 0, GPU: Tesla P4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    512 x 512 x   3 ->  512 x 512 x  32 0.453 BF
   1 conv     64       3 x 3/ 2    512 x 512 x  32 ->  256 x 256 x  64 2.416 BF
   2 conv     64       1 x 1/ 1    256 x 256 x  64 ->  256 x 256 x  64 0.537 BF
   3 route  1 		                           ->  256 x 256 x  64 
   4 conv     64       1 x 1/ 1    256 x 256 x  64 ->  256 x 256 x  64 0.537 BF
   5 conv     32       1 x 1/ 1    256 x 256 x  64 ->  256 x 256 x  32 0.268 BF
   6 conv     64       3 x 3/ 1    256 x 256 x  32 ->  256 x 256 x  64 2.416 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 256 x 256 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    256

## Check the model performance
The metrics run on the test images so they may not be fully representative, only indicative

In [ ]:
!pwd
%cd train 
!cp -rf ../valid/*.* ./

/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files
/content/Vitis-In-Depth-Tutorial/Machine_Learning/Design_Tutorials/07-yolov4-tutorial/dk_files/train


In [ ]:
!./darknet detector map /content/yolotinyv3_medmask_demo/obj.data /content/yolotinyv3_medmask_demo/yolov4-leaky_obj.cfg "/content/darknet/backup/yolov4-leaky_obj_best.weights" -points 0

## Copy best weights to google drive


In [ ]:
!cp /content/darknet/backup/yolov4-leaky_obj_best.weights  '/content/drive/My Drive/'